# Train a Language Model to Detect Human Values in Arguments

The following notebook contains the training-procedure for a single training.
The final Model is an ensemble of several such runs. More information can be found in the system description paper.

In [ ]:
# if on google colab
!pip install -q pytorch-lightning==1.6.4 neptune-client transformers sentencepiece

In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn

from transformers import AutoTokenizer

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import NeptuneLogger

from torchmetrics import AUROC

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import pickle


RANDOM_SEED = 42

pl.seed_everything(RANDOM_SEED)

c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42

If you are training to google colab and want to connect to drive

In [2]:
pl.__version__

'1.6.4'

In [3]:
import os
os.getcwd()
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd ./drive/MyDrive/human_value/human_values_behind_arguments

In [ ]:
!git pull

## Import Modules
We use Pytorch Lightning for the training and therefore import the Lighntning Data and Model Modules....

In [4]:
from data_modules.BertDataModule import BertDataModule, BertDataset
from models.BertFineTunerPl import BertFineTunerPl
from weights.weights import INS #Weights for Weighting Loss Function (optional)
from toolbox.bert_utils import max_for_thres # Algorithm that chooses threshold that maximizes f1-score

### Define Parameters

We define the important params upfront. The object is then logged at neptune.ai

In [5]:
PARAMS = {
    # Language Model and Hyperparameters
    "MODEL_PATH": 'roberta-base',
    "BATCH_SIZE": 8,
    "ACCUMULATE_GRAD_BATCHES": 1,
    "LR": 2e-5,
    "EPOCHS": 3,
    "OPTIMIZER": 'AdamW',
    "DEVICE": torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    "NUM_TRAIN_WORKERS": 4,
    "NUM_VAL_WORKERS": 4,
    "MAX_TOKEN_COUNT":165,
    "RANDOM_SEED": RANDOM_SEED, #Random Seed Selected for this Training Run

    # Apply Weights to loss function (optional, the submitted system does not weight the loss function).
    "WEIGHTS": INS,
    "CRITERION": [nn.BCEWithLogitsLoss()],
    # "CRITERION": [nn.BCEWithLogitsLoss(pos_weight=torch.Tensor(INS))], # Optional

    # Early Stopping Params
    "PATIENCE": 3,
    "VAL_CHECK_INTERVAL": 300,

    # The metric we optimize for. Alternative "custom_f1/Val" and "max"
    "MAX_THRESHOLD_METRIC": "custom", #The f1-score that should maximized (custom = formula for the task evaluation)
    "EARLY_STOPPING_METRIC": "avg_val_loss",
    "EARLY_STOPPING_MODE": "min",

    # Additional Dropout or Additional Hidden Layers (Not used for the final submission)
    "DROPOUT": None, # e.g 0.5 (float)
    "HIDDEN_LAYERS":None, # Of Shape [(512, nn.ReLU()),...] put size of hidden Layer together with activation function in list

    # DATA
    "VALIDATION_SET_SIZE":500,

    "EMBEDDING": "CLS", # "CLS + MEAN" for both. Which information should be used from Bert-Output. CLS Token in Submission.

    "TRAIN_PATH" : "./data/data_training_full.csv", #
    "LEAVE_OUT_DATA_PATH": "./data/leave_out_dataset_300.csv"

}


## Data Loading
Please see the data_generation.ipynb notebook... We create the training-data, and leave-out-datafiles there and save them in the data directory.


In [6]:
train_df = pd.read_csv(PARAMS["TRAIN_PATH"], index_col=0)

We now get the LABEL_COLUMNS

In [7]:
LABEL_COLUMNS = train_df.columns.tolist()[6:]

For the best-performing submission we used a Leave-Out-Dataset, to determine the optimal threshold that maximizes the f1-score at the end. This dataset is used to determine the best threshold for an ensembled model.

In [8]:
leave_out_df = pd.read_csv(PARAMS["LEAVE_OUT_DATA_PATH"], index_col=0)
# train_df, test_df = train_test_split(train_df, test_size=500, random_state=42)

## Model Training

### Linear Learning Rate Schedule
Define Parameters for the Linear Learning Rate Schedule

In [9]:
steps_per_epoch=len(train_df) // PARAMS['BATCH_SIZE']
total_training_steps = steps_per_epoch * PARAMS['EPOCHS']

We'll use a fifth of the training steps for warm-up:

In [10]:
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(515, 2577)

### Prepare Data Modules for the Training

#### Create Validation Set for this Run
To make use of the total available data, the models in the final ensemble are trained on different train-validation splits...

In [11]:
train_df, val_df = train_test_split(train_df, test_size=PARAMS["VALIDATION_SET_SIZE"], random_state=PARAMS["RANDOM_SEED"])

Create Tokenizer and Data Module for the Training

In [12]:
TOKENIZER = AutoTokenizer.from_pretrained(PARAMS["MODEL_PATH"])

In [13]:
data_module = BertDataModule(
    train_df,
    val_df,
    tokenizer=TOKENIZER,
    params=PARAMS,
    label_columns=LABEL_COLUMNS
)

### Create Model

In [14]:
model = BertFineTunerPl(n_classes=len(LABEL_COLUMNS), params=PARAMS, label_columns=LABEL_COLUMNS, n_training_steps=total_training_steps, n_warmup_steps=warmup_steps)
NAME = f"{PARAMS['MODEL_PATH'].replace('/','-')}-BS_{PARAMS['BATCH_SIZE']}-LR_{PARAMS['LR']}-HL_{PARAMS['HIDDEN_LAYERS']}-DROPOUT_{PARAMS['DROPOUT']}"
RUN_ID = None


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We use Neptune.ai to log the experiment. This is optional. You need to create a new project and add the project and api key for the neptune logger.

In [14]:
# Optional: Log Experiments to Neptune

neptune_logger = NeptuneLogger(
    project="Your Project ID", #
    api_key="Your API KEY", #
    name=NAME,
    tags=[f"{PARAMS['MODEL_PATH']}",f"BS-{PARAMS['BATCH_SIZE']}",f"LR-{PARAMS['LR']}",f"LR-{PARAMS['HIDDEN_LAYERS']}"],
    log_model_checkpoints=False
)
neptune_logger.log_hyperparams(PARAMS)

neptune_logger.experiment["train_size"].log(len(train_df))
neptune_logger.experiment["val_size"].log(len(val_df))
RUN_ID = neptune_logger._run_short_id

ModuleNotFoundError: No module named 'neptune.new.utils'

### Add Callbacks and create Pytorch Lightning Trainer

In [15]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename= (f"{RUN_ID}-{NAME}" if RUN_ID else f"{NAME}"),
    save_top_k=1,
    verbose=True,
    monitor=PARAMS["EARLY_STOPPING_METRIC"],
    mode=PARAMS["EARLY_STOPPING_MODE"]
)
early_stopping_callback = EarlyStopping(monitor=PARAMS["EARLY_STOPPING_METRIC"], patience=PARAMS["PATIENCE"], mode=PARAMS["EARLY_STOPPING_MODE"])

trainer = pl.Trainer(
    logger=([neptune_logger] if RUN_ID else []),
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=PARAMS["EPOCHS"],
    # fast_dev_run=True,
    # accelerator="gpu",
    devices=1,
    enable_progress_bar=True,
    val_check_interval=PARAMS["VAL_CHECK_INTERVAL"],
    accumulate_grad_batches=PARAMS["ACCUMULATE_GRAD_BATCHES"],
)

MisconfigurationException: GPUAccelerator can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].

In [18]:
trainer.fit(model, data_module)

c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | bert          | RobertaModel      | 124 M 
1 | hidden_layers | ModuleList        | 0     
2 | classifier    | Linear            | 15.4 K
3 | criterion     | BCEWithLogitsLoss | 0     
----------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
498.644   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 39.36it/s]

c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\core\module.py:491: You called `self.log('val_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\core\module.py:491: You called `self.log('f1_micro_val_threshold', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\core\module.py:491: You called `self.log('f1_macro_val_threshold', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\core\module.py:491: You called

c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   0%|          | 0/797 [00:00<?, ?it/s] 

c:\Users\dschr\Coding\nlp\human_value_detector_tmp\human_value_detector\.venv\lib\site-packages\pytorch_lightning\core\module.py:491: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Epoch 0:  14%|█▍        | 115/797 [02:37<15:36,  0.73it/s, train_loss=0.416]

: 

In [18]:
if RUN_ID:
    with open(f"./checkpoints/{RUN_ID}_PARAMS.pkl", 'wb') as f:
        pickle.dump(PARAMS, f)
else:
    with open(f"./checkpoints/{NAME}_PARAMS.pkl", 'wb') as f:
        pickle.dump(PARAMS, f)

In [ ]:
# If Logging (optional)
neptune_logger.experiment["best_model_checkpoint"].log(trainer.checkpoint_callback.best_model_path)
neptune_logger.log_model_summary(model=model, max_depth=-1)

Now we are done with the training. This process is repeated with several different configurations for the model. More information can be found in the system description paper.

# Evaluation

The predictions for the final submissions are done based on an ensemble.
Hence for ensembling, please continue with the ensemble_eval_and_predict.ipynb notebook.
However, for simplicity or if you are interested, you may want to continue here to evaluate the model performance.

1. We determine the decision threshold to decide when a certain label should be counted as 1, based on the val_data
2. We predict the test_data with it (if splitted above)

We load the model from the best_checkpoint in order to get the model that performed best with respect to the early stopping metric.

In [ ]:
trained_model = BertFineTunerPl.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path,
    params=PARAMS,
    label_columns=LABEL_COLUMNS,
    n_classes=len(LABEL_COLUMNS)
)

trained_model.eval()
trained_model.freeze()

We get the predictions for the val_df.

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

val_dataset = BertDataset(
    val_df,
    tokenizer=TOKENIZER,
    max_token_count=PARAMS["MAX_TOKEN_COUNT"],
    label_columns=LABEL_COLUMNS
)

predictions = []
labels = []

for item in tqdm(val_dataset):
    _, prediction = trained_model(
        item["input_ids"].unsqueeze(dim=0).to(device),
        item["attention_mask"].unsqueeze(dim=0).to(device)
    )
    predictions.append(prediction.flatten())
    labels.append(item["labels"].int())


  0%|          | 0/500 [00:00<?, ?it/s]

Select optimal Threshold on Val Dataset

In [67]:
predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

In [72]:
from toolbox.bert_utils import max_for_thres

In [73]:
THRESHOLD = max_for_thres(y_pred=predictions, y_true=labels, label_columns=LABEL_COLUMNS, average=PARAMS["MAX_THRESHOLD_METRIC"])

Alternatively if you just one to load a model from checkpoint

In [21]:
with open(f'./checkpoints/HCV-409_PARAMS.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
    PARAMS = loaded_dict

trained_model = BertFineTunerPl.load_from_checkpoint(
    "./checkpoints/HCV-409-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt",
    params=PARAMS,
    label_columns=LABEL_COLUMNS,
    n_classes=len(LABEL_COLUMNS)
)

trained_model.eval()
trained_model.freeze()

THRESHOLD = 0.25

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


binarize the predictions with the optimal threshold

In [ ]:
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > THRESHOLD, upper, lower)

In [ ]:
print(f"Threshold: {THRESHOLD}")
print(classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLUMNS,
    zero_division=0,
))

class_rep = classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLUMNS,
    zero_division=0,
    output_dict=True
)

# Use Threshold to predict on Test Data
If we want to predict on the test-data (if you have split it apart, alternatively you could use the leave-out-dataset). For a single Model.

In [22]:
test_df = leave_out_df

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

test_dataset = BertDataset(
    test_df,
    tokenizer=TOKENIZER,
    max_token_count=PARAMS["MAX_TOKEN_COUNT"],
    label_columns=LABEL_COLUMNS
)

predictions = []
labels = []

for item in tqdm(test_dataset):
    _, prediction = trained_model(
        item["input_ids"].unsqueeze(dim=0).to(device),
        item["attention_mask"].unsqueeze(dim=0).to(device)
    )
    predictions.append(prediction.flatten())
    labels.append(item["labels"].int())


predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()


y_pred = predictions.numpy()
y_true = labels.numpy()


###  Binarize the model predictions with Threshold


In [24]:
upper, lower = 1, 0

y_pred = np.where(y_pred > THRESHOLD, upper, lower)

In [25]:
print(f"Threshold: {THRESHOLD}")
print(classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLUMNS,
    zero_division=0,
))

class_rep = classification_report(
    y_true,
    y_pred,
    target_names=LABEL_COLUMNS,
    zero_division=0,
    output_dict=True
)

Threshold: 0.25
                            precision    recall  f1-score   support

   Self-direction: thought       0.41      0.92      0.57        49
    Self-direction: action       0.69      0.89      0.78        75
               Stimulation       0.19      0.64      0.29        14
                  Hedonism       0.20      1.00      0.33         5
               Achievement       0.74      0.92      0.82        86
          Power: dominance       0.39      0.91      0.55        33
          Power: resources       0.50      1.00      0.67        26
                      Face       0.24      0.56      0.34        25
        Security: personal       0.71      0.95      0.81       103
        Security: societal       0.69      0.94      0.80        85
                 Tradition       0.54      1.00      0.70        31
         Conformity: rules       0.75      0.91      0.82        81
 Conformity: interpersonal       0.33      0.82      0.47        11
                  Humility     

In [ ]:
#Logging Optional
neptune_logger.experiment[f"threshold_selected_for_f1_custom_val_opt"].log(THRESHOLD)

for k in class_rep:
    neptune_logger.experiment[f"{k}_precision/Test"].log(class_rep[k]["precision"])
    neptune_logger.experiment[f"{k}_recall/Test"].log(class_rep[k]["recall"])
    neptune_logger.experiment[f"{k}_f1-score/Test"].log(class_rep[k]["f1-score"])
    neptune_logger.experiment[f"{k}_support/Test"].log(class_rep[k]["support"])

# Calculate F1-Score

In [26]:
test_custom_f1 = -1
test_macro_recall = class_rep["macro avg"]["recall"]
test_macro_precision = class_rep["macro avg"]["precision"]
if (test_macro_precision + test_macro_recall) != 0:
    test_custom_f1 = (2*test_macro_recall*test_macro_precision/(test_macro_recall+test_macro_precision))
else:
    test_custom_f1 = 0
print(test_custom_f1)

0.6105788216966658


In [27]:
# Optionally Log

for i, name in enumerate(LABEL_COLUMNS):
    auroc = AUROC(task="binary")
    class_roc_auc = auroc(predictions[:, i], labels[:, i])
    # neptune_logger.experiment[f"{name}_roc_auc/Test"].log(class_roc_auc)

auroc = AUROC(task="multilabel", num_labels=len(LABEL_COLUMNS), average="micro")
total_auroc_micro = auroc(predictions, labels)

auroc = AUROC(task="multilabel", num_labels=len(LABEL_COLUMNS), average="macro")
total_auroc_macro = auroc(predictions, labels)

In [ ]:
# Log Metrics Optionally
neptune_logger.experiment[f"custom_f1/Test"].log(test_custom_f1)
neptune_logger.experiment[f"roc_auc_total_macro/Test"].log(total_auroc_macro)
neptune_logger.experiment[f"roc_auc_total_micro/Test"].log(total_auroc_micro)
neptune_logger.experiment.stop()

## Creating Submission File
Creating the submission file for one Model for the competition. (Note that the submitted systems are ensembles (ensemble_eval_and_predict.ipynb)

In [29]:
test_df_input = pd.read_csv('./data/arguments-test.tsv', sep='\t')

In [30]:
test_df_input["text"] = test_df_input["Premise"]+" " + test_df_input["Stance"]+ " " + test_df_input["Conclusion"]
test_df_input.head()

,Argument ID,Conclusion,Stance,Premise,text
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,affirmative action helps with employment equit...
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,affirmative action can be considered discrimin...
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,naturopathy is very dangerous for the most vul...
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,women shouldn't be in combat because they aren...
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,once eradicated illnesses are returning due to...


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

test_df_dataset = BertDataset(
    data=test_df_input,
    tokenizer=TOKENIZER,
    max_token_count=PARAMS["MAX_TOKEN_COUNT"],
)

predictions = []

for item in tqdm(test_df_dataset):
    _, prediction = trained_model(
        item["input_ids"].unsqueeze(dim=0).to(device),
        item["attention_mask"].unsqueeze(dim=0).to(device)
    )
    predictions.append(prediction.flatten())

predictions = torch.stack(predictions).detach().cpu()

In [26]:
y_pred = predictions.numpy()
upper, lower = 1, 0
y_pred = np.where(y_pred > THRESHOLD, upper, lower)

In [27]:
prediction_dictionary = {}
prediction_dictionary["Argument ID"] = test_df_input["Argument ID"]
for idx, l_name in enumerate(LABEL_COLUMNS):
  prediction_dictionary[l_name]=y_pred[:,idx]

test_prediction_df = pd.DataFrame(prediction_dictionary)
test_prediction_df.head()

,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A26004,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,A26010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,A26016,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,1,0,0,1
3,A26024,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,A26026,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
if RUN_ID:
    test_prediction_df.to_csv(f"submissions/{RUN_ID}-submission_test.txt", sep="\t", index=False)
else:
    test_prediction_df.to_csv(f"submissions/{NAME}-submission_test.txt", sep="\t", index=False)

# Looking at single predictions

In [35]:
def print_example_prediction(record, show_all_probs=False, THRESHOLD=0.3):

    print(record["Argument ID"])
    print(record["text"])
    print(f"True Label: {record.category}")


    encoding = TOKENIZER.encode_plus(
        record.text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding="max_length",
        return_attention_mask=True,
        return_tensors='pt',
    )

    _, test_prediction = trained_model(encoding["input_ids"], encoding["attention_mask"])
    test_prediction = test_prediction.flatten().numpy()

    res = {}
    if show_all_probs:
        for label, prediction in zip(LABEL_COLUMNS, test_prediction):
            print(f"{label}: {prediction}")
            res[label] = prediction

    else:
        print(f"Predictions:")
        for label, prediction in zip(LABEL_COLUMNS, test_prediction):
            if prediction < THRESHOLD:
                continue
            print(f"{label}: {prediction}")
            res[label] = prediction
    return res

In [36]:
# 13 whaling is good one
trained_model.to("cpu")
test_record = test_df.iloc[6]
print_example_prediction(test_record, show_all_probs=False, THRESHOLD=THRESHOLD)


A18309
social media gives it users a place to seek support when in need whether emotional or financially, things that would be more difficult if not impossible to do outside of their home. against Social media brings more harm than good
True Label: ['Self-direction: action', 'Face', 'Security: personal', 'Benevolence: caring', 'Benevolence: dependability']
Predictions:
Self-direction: action: 0.49473991990089417
Stimulation: 0.40371981263160706
Hedonism: 0.4516661763191223
Security: personal: 0.9821780323982239
Benevolence: caring: 0.9349980354309082
Universalism: tolerance: 0.327671617269516


{'Self-direction: action': 0.49473992,
 'Stimulation': 0.4037198,
 'Hedonism': 0.45166618,
 'Security: personal': 0.98217803,
 'Benevolence: caring': 0.93499804,
 'Universalism: tolerance': 0.32767162}